# Содержание
1. Описание проекта
2. Описание задачи
3. Возможные методы
4. Описание spring rank
5. Процесс формирования списка
6. Описание результата
7. Дальнейшие задачи

# Описание проекта (TBD)

# Описание задачи (TBD)

# Возможные методы (TBD)

# Описание SpringRank

SpringRank решает задачу ранжирования вершин ориентированного мультиграфа, ранги представлены вещественными  числами. Этот алгоритм предполагает нахождение рангов через минимизацию энергии системы пружин, коэффициенты упругости которых пропорциональны весам дуг, а сами пружины имеют одинаковую длину и связывают точки, единственная координата которых соответствует рангу.

Предположение выше можно записать суммой энергии всех пружин:
$$H(s) = \frac{1}{2}\sum_{i,j} kx^2 = \frac{1}{2}\sum_{i,j} A_{ij}(s_i - s_j - 1)^2$$
Минимумы этого выражения находятся в точках, где производная равна нулю, т.е. являются решениями следующей системы уравнений:
$$(D^{out} + D^{in} - (A + A^T))s = (D^{out} - D^{in})1 \tag{1}$$
Где $D^{in}, D^{out}$ - диагональные матрицы полустепеней захода и исхода вершин соответственно, $1$ - вектор, состоящий из единиц, а $А$ - матрица смежности графа.

### Регуляризация
Стоит отметить, что, исходя из определения, матрица $(D^{out} + D^{in} - (A + A^T))$ имеет ранг $r \leq n - 1$ (неравенство строгое, если граф не связан), и существует прямая, на которой лежат все решения системы уравнений (1). В статье предлагается выбирать то решение, в котором один из рангов $s_n = 0$, однако в результате при незначительном изменении графа новые ранги могут сильно отличаться. Пример приведен ниже ... . Проблема актуальна в случае, если граф не имеет статистически значимой иерархической структуры. 

Для устранения этой проблемы в статье предлагается применение L2 - регуляризации, которая преобразует выражение (1) в:
$$(D^{out} + D^{in} - (A + A^T + \alpha E))s = (D^{out} - D^{in})1$$
Матрица коэффициентов в правой части при коэффициенте регуляризации $\alpha > 0$ становится обратимой, что делает возможным существование только одного локального минимума. Однако при больших $\alpha$ единственным верным решением становится $s = 0$, поэтому важно подобрать коэффициент регуляризации так, чтобы все еще можно было использовать вычисленные ранги для предсказания количества и направлений дуг. Для этого предлагается использовать процедуру кросс-валидации.

# Процесс формирования списка

При формировании списка генезиса количество участников сети и количество распределяемой между ними репутации - конечные значения, поэтому начальное распределение предполагало выбор адресов в сети Ethereum, которые, скорее всего, являются локальными "лидерами мнений". Так далее будем называть адреса, которые в плотно связанном подграфе (сообществе) связаны с наибольшим количеством вершин в нем.

Было предложено составлять этот список на основе меры центральности вершин графа транзакций. В качестве алгоритма ранжирования было решено использовать описанный выше SpringRank. Описанная в статье об этом алгоритме генеративная модель позволяет использовать значения рангов для вычисления количества и направления дуг между вершинами по формулам:

$$E_{ij} = c \exp{-\frac{\beta}{2}(s_i - s_j - 1)^2}$$

$$p_{ij} = \frac{1}{1 + \exp-2\beta(s_i - s_j)}$$

Т.е. количество дуг между вершинами $E_{ij}$ увеличивается при уменьшении разницы их рангов, а количество дуг от вершины с меньшим рангом меньше, чем количество дуг в обратном направлении (эти значения равны соответственно $E_{ij}p_{ij}$ и $E_{ij}(1 - p_{ij})$)

Этими свойствами можно воспользоваться при составлении списка для начального распределения. Действительно, предполагаемые "лидеры мнений" по условию имеют максимальное количество дуг в некотором подграфе, а это значило бы наличие у соотвествующих им вершин близких рангов. Первое решение, которое приходит в голову - провести оценку плотности рангов, выделить участки максимальной плотности (кластеры) и в итоговый список помещать адреса из каждого кластера, имеющие максимальную плотность в окрестности соответствующего им ранга. 

Кластеризацию небольшого количества рангов можно провести алгоритмом MeanShift, который хорошо подходит для кластеризации одномерных данных при отсутствии фиксированного количества кластеров. Однако в графе транзакций Ethereum находится ~ 40M адресов, и цифра настолько велика для временной сложности этого алгоритма (которая оценивается как  $O(n^2)$  при ограничении числа итераций, где  $n$  - количество точек), что сильно увеличивает количество выпитых чашек кофе между запуском и получением результата.

Вместо MeanShift был предложен способ нахождения локальных экстремумов, который подразумевал простой поиск минимальных и максимальных значений в пределах некоторого скользящего окна для значений оценки плотности. Кластером в этом случае можно считать отрезок между двумя локальными минимумами, содержащий локальный максимум. Существуют реализации используемого в этом случае алгоритм оценки плотности KDE с временной сложностью  $O(n)$ , что применимо в данном случае

Важно отметить, что перед поиском экстремумов необходимо линейно интерполировать значения KDE с фиксированным шагом сетки, поскольку в исходной оценке плотности в окрестности локального максимума лежит гораздо большее количество точек, нежели в окрестности локального минимума, что делает затруднительным выбор размера скользящего окна.

# Описание результата

ТОП-20k всех участников распределения получил более 80% распределенной репутации. Процент распределенной репутации показан на рисунке ниже.

![](./images/top.png)

Текущий список содержит ~1М адресов, выбранных с помощью процедуры, описанной выше. За все время этими адресами было сделано 11.5M транзакций, что составляет 9.4% от всех транзакций, сделанных не контрактами. Наиболее активные адреса представлены в таблице ниже. 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>percentage</th>
      <th>transactions</th>
    </tr>
    <tr>
      <th>address</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98</th>
      <td>0.000027</td>
      <td>642978</td>
    </tr>
    <tr>
      <th>0xc6300135a8fcd43123bb486ff06831b5345d0971</th>
      <td>0.000177</td>
      <td>453387</td>
    </tr>
    <tr>
      <th>0x6cc5f688a315f3dc28a7781717a9a798a59fda7b</th>
      <td>0.000011</td>
      <td>435743</td>
    </tr>
    <tr>
      <th>0x5dcaa1d8d8132e5bf9cf12deccfc0cecf26a780d</th>
      <td>0.000049</td>
      <td>202782</td>
    </tr>
    <tr>
      <th>0xfd54078badd5653571726c3370afb127351a6f26</th>
      <td>0.000056</td>
      <td>172438</td>
    </tr>
    <tr>
      <th>0x343295b49522cfc38af517c58ebb78565c42ed95</th>
      <td>0.000050</td>
      <td>163688</td>
    </tr>
    <tr>
      <th>0x4b01721f0244e7c5b5f63c20942850e447f5a5ee</th>
      <td>0.000025</td>
      <td>153769</td>
    </tr>
    <tr>
      <th>0x18916e1a2933cb349145a280473a5de8eb6630cb</th>
      <td>0.000055</td>
      <td>144264</td>
    </tr>
    <tr>
      <th>0x5baeac0a0417a05733884852aa068b706967e790</th>
      <td>0.000005</td>
      <td>135305</td>
    </tr>
    <tr>
      <th>0xb2a48f542dc56b89b24c04076cbe565b3dc58e7b</th>
      <td>0.000050</td>
      <td>127079</td>
    </tr>
  </tbody>
</table>

Список содержит 32 адреса, которые присутствовали в генезисе Ethereum, это составляет 5.9% от всех участников генезиса, которые были активны в последний год. Некоторые адреса генезиса, попавшие в список, представлены в таблице ниже.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>percentage</th>
      <th>eth</th>
    </tr>
    <tr>
      <th>address</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0x3bf86ed8a3153ec933786a02ac090301855e576b</th>
      <td>5.214880e-08</td>
      <td>450000.000</td>
    </tr>
    <tr>
      <th>0x537f9d4d31ef70839d84b0d9cdb72b9afedbdf35</th>
      <td>7.491039e-06</td>
      <td>70000.000</td>
    </tr>
    <tr>
      <th>0x17125b59ac51cee029e4bd78d7f5947d1ea49bb2</th>
      <td>2.666275e-06</td>
      <td>22000.000</td>
    </tr>
    <tr>
      <th>0x36d85dc3683156e63bf880a9fab7788cf8143a27</th>
      <td>6.220137e-06</td>
      <td>20000.000</td>
    </tr>
    <tr>
      <th>0x97b91efe7350c2d57e7e406bab18f3617bcde14a</th>
      <td>3.038474e-06</td>
      <td>9999.980</td>
    </tr>
    <tr>
      <th>0x5aaf1c31254a6e005fba7f5ab0ec79d7fc2b630e</th>
      <td>4.872832e-05</td>
      <td>5910.000</td>
    </tr>
    <tr>
      <th>0xf5437e158090b2a2d68f82b54a5864b95dd6dbea</th>
      <td>5.635966e-06</td>
      <td>4010.732</td>
    </tr>
    <tr>
      <th>0x32fa0e86cd087dd68d693190f32d93310909ed53</th>
      <td>2.575433e-06</td>
      <td>4000.000</td>
    </tr>
    <tr>
      <th>0x08306de51981e7aca1856859b7c778696a6b69f9</th>
      <td>1.502717e-08</td>
      <td>3200.000</td>
    </tr>
    <tr>
      <th>0xcd0b0257e783a3d2c2e3ba9d6e79b75ef98024d4</th>
      <td>1.091029e-06</td>
      <td>2945.500</td>
    </tr>
  </tbody>
</table>

Список содержит 50 адресов, которые являются майнерами или майнинг пулами в Ethereum, это составляет 4.3% от всех майнеров, активных в последний год. Наиболее активные адреса майнеров, попавшие в список, представлены в таблице ниже.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>percentage</th>
      <th>blocks</th>
    </tr>
    <tr>
      <th>address</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0x441d463d69cb6eaa851eb6138c3efac95c86d17b</th>
      <td>7.129562e-08</td>
      <td>6488</td>
    </tr>
    <tr>
      <th>0xa855c20a1351acd2690c716e2709c7dff3978d12</th>
      <td>1.683240e-06</td>
      <td>3051</td>
    </tr>
    <tr>
      <th>0x9dd134d14d1e65f84b706d6f205cd5b1cd03a46b</th>
      <td>4.643750e-07</td>
      <td>1466</td>
    </tr>
    <tr>
      <th>0xb6cf40aee9990c25d7d6193952af222e120b31c2</th>
      <td>7.160081e-08</td>
      <td>683</td>
    </tr>
    <tr>
      <th>0x3e8702685ed7826a55acb1c6eb4a6d74638d165c</th>
      <td>1.394298e-04</td>
      <td>669</td>
    </tr>
    <tr>
      <th>0x11905bd0863ba579023f662d1935e39d0c671933</th>
      <td>1.608024e-04</td>
      <td>430</td>
    </tr>
    <tr>
      <th>0x4947190fcc30d3fb3a16233b259bfe6b3492339a</th>
      <td>7.071169e-08</td>
      <td>400</td>
    </tr>
    <tr>
      <th>0xf78465bce3c4620fd124c67d523d2ab80a76c0d8</th>
      <td>2.835992e-08</td>
      <td>369</td>
    </tr>
    <tr>
      <th>0x3b670df9fbc5866ee356b426361d57911f1da682</th>
      <td>6.872996e-07</td>
      <td>194</td>
    </tr>
    <tr>
      <th>0x538ee32f26672fefe6b6732bb664a8637ed30a19</th>
      <td>1.461556e-06</td>
      <td>161</td>
    </tr>
  </tbody>
</table>

Список содержит 18559 адресов, которые участвовали в 65 событиях распределения ERC-20 токенов (в основном - эйрдропы и ICO) в Ethereum. Таблица распределений токенов и процента адресов, попавших в список, среди активных участников, представлена ниже.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>percentage</th>
      <th>name</th>
    </tr>
    <tr>
      <th>token</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0xf0df09387693690b1e00d71eabf5e98e7955cff4</th>
      <td>66.666667</td>
      <td>ENDO.network Promo Token</td>
    </tr>
    <tr>
      <th>0x1a8f615f0dd39b9de8ad26db89cfa76f7c9d0274</th>
      <td>41.463415</td>
      <td>UTour</td>
    </tr>
    <tr>
      <th>0xd4ae0807740df6fbaa7a258907132a2ac8d52fbc</th>
      <td>12.845953</td>
      <td>KEOSToken</td>
    </tr>
    <tr>
      <th>0xa720911a58d948fc9328560aecfc51e907404fc4</th>
      <td>11.764706</td>
      <td>FOR Agriculture Coin</td>
    </tr>
    <tr>
      <th>0xf2eab3a2034d3f6b63734d2e08262040e3ff7b48</th>
      <td>10.559006</td>
      <td>CANDY</td>
    </tr>
    <tr>
      <th>0xd4de05944572d142fbf70f3f010891a35ac15188</th>
      <td>9.969789</td>
      <td>Bulleon Promo Token</td>
    </tr>
    <tr>
      <th>0x7b2f9706cd8473b4f5b7758b0171a9933fc6c4d6</th>
      <td>9.926082</td>
      <td>An Etheal Promo</td>
    </tr>
    <tr>
      <th>0x4f4d22ca77222ae3d51e308c9a8f0e564f98e77a</th>
      <td>9.615385</td>
      <td>Bulleon Promo Token</td>
    </tr>
    <tr>
      <th>0xbc8deee89f1cf4b661514185aa1ab780336c4c4a</th>
      <td>9.589041</td>
      <td>Poker IO</td>
    </tr>
    <tr>
      <th>0xeb021dd3e42dc6fdb6cde54d0c4a09f82a6bca29</th>
      <td>9.055877</td>
      <td>FNKOSToken</td>
    </tr>
  </tbody>
</table>

# Дальнейшие задачи

Среди предложенных к рассмотрению задач есть те, без выполнения которых любые изменения, внесенные в процедуру формирования списка, не будут иметь никакого смысла. Такие задачи отмечены **жирным** шрифтом

- **Уточнение предсказательной способности модели**

Было проверено, что иерархическая структура, выделенная в графе транзакций Ethereum, может не быть статистически значимой. Предлагается провести кросс-валидацию и выяснить, с какой точностью возможно предсказание дуг на основе вычисленных рангов. Для улучшения результата возможны либо применение регуляризации, либо выбор другого метода ранжирования (в крайнем случае - реструктуризация исходного графа). 

- **Уточнение метрик качества списка**

При формировании списка важно понимание того, как он отличается от набора случайно выбранных адресов. В качестве такой метрики помимо приведенных выше (доля сделанных транзакций, количество участников генезиса, количество майнеров, количество активных участников других распределений) предлагается использовать охват с поправкой на вероятность отправки транзакции - предлагается к обсуждению.

- "Распутывание" или удаление крупных посредников

Из-за наличия в списке адресов бирж и некоторых других приложений, которые являются посредниками при переводе средств между адресами, теряется информация о настоящих взаимодействиях между адресами. Эту ситуацию наблюдаем с адресом Димы, который, при наличии сравнительно большого количества значимых транзакций среди остальных адресов в соответствующем кластере, оказался около одного из адресов Bittrex и более крупных трейдеров, которые вытеснили его из списка. 

Желательно исключить адреса-посредники из рассмотрения (для централизованных бирж) или добавить в граф транзакции напрямую между взаимодействующими через посредника узлами (для децентрализованных бирж).

- Изменение способа выбора адресов

Способ прореживания списка, основанный на выборе адресов с ближайшими рангами в своем кластере, является очевидным, но далеко не лучшим способом выбора "лидеров мнений". При построении соответствующих графиков можно видеть, что крупные адреса часто не находятся в центре своих кластеров. Возможно, имеет смысл использовать ранги только для кластеризации адресов, а на подграфе каждого кластера использовать другой алгоритм.

- Добавление в граф новой информации

На данный момент не используется информация о количестве переведенного между адресами эфира или токенов. 